# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [4]:
data.columns

Index(['CustomerID', 'FirstName', 'LastName', 'SalesID', 'ProductID',
       'ProductName', 'Quantity'],
      dtype='object')

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
df = data[["CustomerID", "ProductName", "Quantity", "FirstName", "LastName"]]
purchase = data.groupby(["CustomerID", "ProductName", "FirstName", "LastName"]).sum()
#purchase["CustomerName"] = data["FirstName"] + " " +  data["LastName"]
display(purchase.head())

SalesID  \
CustomerID ProductName                   FirstName LastName            
33         Apricots - Dried              Lindsay   Santana    172592   
           Assorted Desserts             Lindsay   Santana   5363705   
           Bandage - Flexible Neon       Lindsay   Santana   6299165   
           Bar Mix - Pina Colada, 355 Ml Lindsay   Santana   3147536   
           Beans - Kidney, Canned        Lindsay   Santana   6507033   

                                                             ProductID  \
CustomerID ProductName                   FirstName LastName              
33         Apricots - Dried              Lindsay   Santana         324   
           Assorted Desserts             Lindsay   Santana         219   
           Bandage - Flexible Neon       Lindsay   Santana         273   
           Bar Mix - Pina Colada, 355 Ml Lindsay   Santana         281   
           Beans - Kidney, Canned        Lindsay   Santana         300   

                                                             Quantity  
CustomerID ProductName                   FirstName LastName            
33         Apricots - Dried              Lindsay   Santana          1  
           Assorted Desserts             Lindsay   Santana          1  
           Bandage - Flexible Neon       Lindsay   Santana          1  
           Bar Mix - Pina Colada, 355 Ml Lindsay   Santana          1  
           Beans - Kidney, Canned        Lindsay   Santana          1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [9]:
matrix = pd.pivot_table(df, values= "Quantity", index="ProductName", columns="CustomerID", dropna=True).fillna(0)
display(matrix.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [11]:
from scipy.spatial.distance import pdist, squareform

squareform(pdist(matrix.T, 'euclidean'))
distances = pd.DataFrame(1/(1 + squareform(pdist(matrix.T, 'euclidean'))), 
                         index=matrix.columns, columns=matrix.columns)

display(distances.head())
simil_customer = pdist(matrix.T, "euclidean")
display(simil_customer)

squareform(simil_customer)


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903


array([ 10.72380529,   9.64365076,   9.89949494, ..., 266.9269563 ,
       266.9269563 , 269.25824036])

array([[  0.        ,  10.72380529,   9.64365076, ..., 197.94948851,
        218.8241303 , 203.79892051],
       [ 10.72380529,   0.        ,  10.67707825, ..., 197.34994299,
        218.62525014, 203.95342606],
       [  9.64365076,  10.67707825,   0.        , ..., 197.32460566,
        218.02981447, 203.80628057],
       ...,
       [197.94948851, 197.34994299, 197.32460566, ...,   0.        ,
        266.9269563 , 266.9269563 ],
       [218.8241303 , 218.62525014, 218.02981447, ..., 266.9269563 ,
          0.        , 269.25824036],
       [203.79892051, 203.95342606, 203.80628057, ..., 266.9269563 ,
        269.25824036,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [24]:

variable = distances[200].sort_values(ascending=False)[1:6]
top_5 = variable.index.to_list()
top_5

#top_5 = similarities.index.tolist()

[1920, 1072, 3253, 3317, 3909]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [28]:

df_aux = purchase.reset_index()
simil_custo = df_aux[df_aux["CustomerID"].isin(top_5)]

display(simil_custo.head())

,CustomerID,ProductName,FirstName,LastName,SalesID,ProductID,Quantity
1082,1072,"Appetizer - Mini Egg Roll, Shrimp",Guillermo,Leach,4770059,327,1
1083,1072,Appetizer - Sausage Rolls,Guillermo,Leach,6604526,73,1
1084,1072,"Beans - Kidney, Canned",Guillermo,Leach,1148109,300,1
1085,1072,Beef - Montreal Smoked Brisket,Guillermo,Leach,705042,373,1
1086,1072,Beef - Striploin Aa,Guillermo,Leach,1804237,339,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [32]:
list_productName = simil_custo["ProductName"].tolist()

agrupated = simil_custo.groupby(["ProductName"])["Quantity"].sum().sort_values(ascending=True)
display(agrupated.head())

ProductName
Appetizer - Mini Egg Roll, Shrimp    1
Pasta - Angel Hair                   1
Pasta - Penne, Rigate, Dry           1
Pastry - Choclate Baked              1
Peas - Pigeon, Dry                   1
Name: Quantity, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.